# pretrained_models

Download and run pretrained models on the BuildingsBench benchmark.

This tutorial assumes you have followed the [installation instructions](https://nrel.github.io/BuildingsBench/#installation) for BuildingsBench, have [downloaded the datasets](https://nrel.github.io/BuildingsBench/datasets), and have set the `$BUILDINGS_BENCH` environment variable appropriately.

## Step 1: Download a pretrained model from the OEDI

From the base directory of the BuildingsBench code repository, run the following in the command line:

```bash
mkdir checkpoints
cd checkpoints
wget https://oedi-data-lake.s3.amazonaws.com/buildings-bench/v1.1.0/checkpoints/Transformer_Gaussian_S.pt
```

Other pretrained models are available at `https://oedi-data-lake.s3.amazonaws.com/buildings-bench/v1.1.0/checkpoints/`.

## Step 1.5: Test that your model is working

You should be able to run the following command from the base directory of the BuildingsBench code repository (assuming you are running on a GPU):

```bash
python3 scripts/zero_shot.py --model TransformerWithGaussian-S --checkpoint ./checkpoints/Transformer_Gaussian_S.pt
```

In what follows, we explain how to load and run the pretrained models in more detail.

## Step 2: Load the pretrained model

In [1]:
# global variables for this tutorial

# Set this to your BuildingsBench github repo path
repo_path = ''
# This has to match the name of the .tomli file in the configs folder
model_name = 'TransformerWithGaussian-S'
# path to the checkpoint file you downloaded
checkpoint = f'{repo_path}/checkpoints/Transformer_Gaussian_S.pt'
# device, either 'cpu' or 'cuda:0'
device = 'cuda:0'
# dataset name
dataset = 'electricity'
# 'boxcox' if using the Gaussian model (continuous load values) else ''
scaler_transform = 'boxcox'


In [2]:
import tomli
import os
import torch
from pathlib import Path 
from buildings_bench.models import model_factory

# Load the model arguments from the .tomli file
config_path = Path(f'{repo_path}/buildings_bench/configs')
if (config_path / f'{model_name}.toml').exists():
    toml_args = tomli.load(( config_path / f'{model_name}.toml').open('rb'))
    model_args = toml_args['model']
else:
    raise ValueError(f'Config {model_name}.toml not found.')

# Create the model and move it to the device
model, _, predict = model_factory(model_name, model_args)
model = model.to(device)

# Load from the checkpoint
model.load_from_checkpoint(checkpoint)
model.eval()

LoadForecastingTransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.0, inplace=False)
          (dropout2): Dropout(p=0.0, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerDecoderLayer(
          (self_attn): Multihe

## Step 3: Load a building time series

In [4]:
from buildings_bench import load_torch_dataset
from buildings_bench.tokenizer import LoadQuantizer

transform_path = Path(os.environ.get('BUILDINGS_BENCH', '')) \
    / 'metadata' / 'transforms'

# Load the dataset generator
buildings_datasets_generator = load_torch_dataset('electricity',
                                                  apply_scaler_transform=scaler_transform,
                                                  scaler_transform_path=transform_path)

# the `building_dataset` is a torch.utils.data.Dataset object
for building_id, building_dataset in buildings_datasets_generator:
    break

## Step 4: Forward and inverse transforms for pre-processing the load time series

The pretrained models expects the load time series to be transformed before being passed to the model.
There are two main ways we pre-process the load time series values in BuildingsBench.

1. The transformer trained with continuous load values and which predicts a Gaussian distribution per time step (e.g., `Transformer_Gaussian_S.pt`) uses a [Box-Cox transform](https://nrel.github.io/BuildingsBench/API/utilities/buildings_bench-transforms/#boxcoxtransform) to normalize the data. In our `buildings_bench` library, the forward Box-Cox transform is applied to the time series in the dataloader when creating a mini-batch. The inverse Box-Cox transform must be called manually on the predicted values to convert them back to the original scale (`buildings_bench.transforms.BoxCoxTransform.undo_transform`).
2. The transformer trained with discrete load values and which predicts a categorical distribution per time step (e.g., `Transformer_Tokens_S.pt`) uses a KMeans [LoadQuantizer](https://nrel.github.io/BuildingsBench/API/utilities/buildings_bench-tokenizer/#tokenizer-quick-start) to tokenize the continuous load values into discrete values. In our `buildings_bench` library, the forward tokenization transform is applied directly before calling the model's forward pass, as we are able to run KMeans quantization on the GPU via `faiss-gpu` which vastly accelerates this step (`buildings_bench.tokenizer.LoadQuantizer.transform/undo_transform`).

Here is our code for setting up the data transforms for the pretrained models:

In [7]:
if scaler_transform == '':
    # The tokenizer 
    load_transform = LoadQuantizer(
        with_merge=True,
        num_centroids=model.vocab_size,
        device='cuda:0' if 'cuda' in device else 'cpu')
    load_transform.load(transform_path)
    # Grab the forward and inverse transform (tokenization) functions
    transform = load_transform.transform
    inverse_transform = load_transform.undo_transform
elif scaler_transform != '': # continuous values
    # the forward transform is handled by the Dataset 
    # so make this an identity function
    transform = lambda x: x 

    if isinstance(building_dataset, torch.utils.data.ConcatDataset):
        inverse_transform = building_dataset.datasets[0].load_transform.undo_transform
    else:            
        # Grab the undo transform function from the dataset object
        inverse_transform = building_dataset.load_transform.undo_transform


## Step 5: Forward pass

In [8]:
# create a dataloader for the building,
# this will extract min(len(building_dataset),360) x [context_len + horizon] windows
# in a sliding window fashion from the building
building_dataloader = torch.utils.data.DataLoader(
                        building_dataset,
                        batch_size=360,
                        shuffle=False)

for batch in building_dataloader:

    for k,v in batch.items():
        batch[k] = v.to(device)

    continuous_load = batch['load'].clone()
    continuous_targets = continuous_load[:, model.context_len:]

    # Apply forward transform
    batch['load'] = transform(batch['load'])

    # These could be tokens or continuous values
    targets = batch['load'][:, model.context_len:]

    # Call the model with the predict (for eval only)
    if device == 'cuda:0':
        with torch.cuda.amp.autocast():
            predictions, distribution_params = predict(batch)
    else:
        predictions, distribution_params = predict(batch)

## Step 6: Inverse transform

To get the predicted load time series back into the original scale, we need to apply the inverse transform to the predicted values. This is a bit complicated because the inverse transform is different for the two pretrained models and we might also want to invert the probabilistic parameters (e.g., the mean and variance of the Gaussian distribution) back into the original scale. The latter is unnecessary for the discrete transformer.

In [9]:
# Either "detokenizes" the discrete load tokens back to continuous 
# values, or undoes the Box-Cox transform on the continuous values
predictions = inverse_transform(predictions)

# Backproject the Gaussian params to an approximate Gaussian in unscaled space
# See our paper for details
if scaler_transform == 'boxcox':   
    mu = inverse_transform(distribution_params[:,:,0])
    muplussigma = inverse_transform(torch.sum(distribution_params,-1))
    sigma = muplussigma - mu
    muminussigma = inverse_transform(distribution_params[:,:,0] - distribution_params[:,:,1])
    sigma = (sigma + (mu - muminussigma)) / 2
    distribution_params = torch.cat([mu.unsqueeze(-1), sigma.unsqueeze(-1)],-1)


In [10]:
print(predictions)

tensor([[[2.1387],
         [2.1055],
         [2.4180],
         ...,
         [2.0273],
         [1.8086],
         [2.1543]],

        [[2.1055],
         [1.9512],
         [2.3086],
         ...,
         [2.1543],
         [2.1719],
         [2.2383]],

        [[2.1543],
         [2.0586],
         [2.0898],
         ...,
         [2.1875],
         [2.2383],
         [2.2910]],

        ...,

        [[2.1055],
         [2.2051],
         [2.2383],
         ...,
         [1.4023],
         [1.2637],
         [1.5918]],

        [[2.6738],
         [2.5723],
         [2.5117],
         ...,
         [2.4355],
         [2.4551],
         [2.5723]],

        [[2.6934],
         [2.2910],
         [2.5117],
         ...,
         [2.4180],
         [2.5117],
         [2.6738]]], device='cuda:0', dtype=torch.float16)
